In [1]:
!pip install -r requirements.python.txt --user
!python -m spacy download en
!python -m spacy link en_core_web_sm en_default
import pandas as pd
import py7zr
from sentence_processing import process_sentence

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[!] As of spaCy v3.0, model symlinks are not supported anymore. You can load
trained pipeline packages using their full names or from a directory path.


In [2]:
# Extract tsv files
with py7zr.SevenZipFile('data/train.tsv.7z', mode='r') as z:
    z.extract(path="data")
with py7zr.SevenZipFile('data/test.tsv.7z', mode='r') as z:
    z.extract(path="data")

In [3]:
full_df = pd.read_csv("data/train.tsv", sep='\t')

In [4]:
data_df = full_df.drop('shipping',axis=1)
data_df = data_df.dropna()
# Only using the instances reuired for test/train
data_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,item_description
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,Adorable top with a hint of lace and a key hol...
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,Size small but straps slightly shortened to fi...
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,You get three pairs of Sophie cheer shorts siz...
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,Girls Size small Plus green. Three shorts total.


In [5]:
for col in data_df:
    print(col," : ", data_df[col].nunique())

train_id  :  846982
name  :  695132
item_condition_id  :  5
category_name  :  914
brand_name  :  4791
price  :  796
item_description  :  751827


In [6]:
# Combining descriptive data to one column
data_df['description'] = data_df[['name', 'brand_name', 'item_description', 'category_name']].agg(' '.join, axis=1)
data_df = data_df.drop(['name', 'brand_name', 'item_description', 'train_id', 'category_name'], axis=1)
data_df['description'] = data_df['description'].str.split()
data_df

,item_condition_id,price,description
1,3,52.0,"[Razer, BlackWidow, Chroma, Keyboard, Razer, T..."
2,1,10.0,"[AVA-VIV, Blouse, Target, Adorable, top, with,..."
6,3,64.0,"[Acacia, pacific, tides, santorini, top, Acaci..."
7,3,6.0,"[Girls, cheer, and, tumbling, bundle, of, 7, S..."
8,3,19.0,"[Girls, Nike, Pro, shorts, Nike, Girls, Size, ..."
...,...,...,...
1482525,2,7.0,"[Red, Victoria's, Secret, bra, w, sequins, PIN..."
1482528,2,18.0,"[Victoria's, Secret, Tankini, Sz., Large, Vict..."
1482529,2,34.0,"[Men's, UA, [rm], Under, Armour, [rm], for, th..."
1482530,2,20.0,"[Free, People, Inspired, Dress, Free, People, ..."


In [7]:
data_df = process_sentence(data_df, 'description', feat=2000)
data_df

,item_condition_id,price_x,00,10,100,11,12,13,14,15,...,yoga,york,you,youniqu,your,youth,zara,zip,zipper,zoom
1,3,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846975,3,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846976,1,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846978,3,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846980,2,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
label = data_df['price_x'].copy()
features = data_df.drop('price_x', axis=1)
features

,item_condition_id,00,10,100,11,12,13,14,15,16,...,yoga,york,you,youniqu,your,youth,zara,zip,zipper,zoom
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846975,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846976,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846978,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846980,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.15, random_state=42)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


# Identifying each attribute for pipeline use
num_attribs = []
cat_attribs = []

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),   #returns a dense matrix
    ('cat', OneHotEncoder(), cat_attribs) #returns a sparse matrix
])

In [11]:
full_pipeline = full_pipeline.fit(X_train)
X_train_prep = full_pipeline.transform(X_train)
X_test_prep = full_pipeline.transform(X_test)

In [14]:
!pip install keras
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached libclang-13.0.0-py2.py3-none-win_amd64.whl (13.9 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cach

In [16]:
def run_model1(X_train, y_train, X_valid, y_valid):
    '''- returns an MLP model trained on tfidf vectorized sparse input.
    - Does not perform best on binarized input.
    - Uses Adam optimizer with constant learning rate. 
    - trains 2 epochs, Batch size is doubled at every epoch to speed up the optimization'''

    model_in = ks.Input(shape=(X_train.shape[1],), dtype='float32', sparse=True)
    out = ks.layers.Dense(256, activation='relu')(model_in)
    # out = ks.layers.Dropout(0.1)(out)     ## performance is better without dropouts
    out = ks.layers.Dense(64, activation='relu')(out)
    # out = ks.layers.Dropout(0.1)(out)
    out = ks.layers.Dense(64, activation='relu')(out)
    # out = ks.layers.Dropout(0.2)(out)
    out = ks.layers.Dense(32, activation='relu')(out)
    out = ks.layers.Dense(1)(out)
    model = ks.Model(model_in, out)
    
    model.compile(loss='mean_squared_error', optimizer=ks.optimizers.Adam(lr=3e-3))
    for i in range(2):
        with timer(f'epoch {i + 1}'):
            model.fit(x=X_train, y=y_train, batch_size=2**(9 + i), epochs=1, verbose=1, validation_data=(X_valid, y_valid))
    
    return model

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [23]:
model3 = run_model1(X_train, y_train, X_valid, y_valid)

AttributeError: module 'keras.optimizers' has no attribute 'Adam'